# **RAG (Retrieval-Augmented Generation) Systems**

##  **1. What is a RAG System?**

- A *Retrieval-Augmented Generation (RAG)* system is a type of AI system that combines large language models (LLMs) with external information sources (like documents, databases, or knowledge bases). Instead of relying purely on the knowledge baked into the LLM, a RAG system retrieves relevant information from external sources and uses it to generate more accurate, up-to-date, and contextually relevant responses.

**Why do we need RAG?**

LLMs have limitations:

- Knowledge cutoff: They might not know recent events.
- Hallucinations: They sometimes generate false or misleading information.
- Limited memory: They can only “remember” so much context within a conversation.

RAG allows LLMs to access external knowledge, improving:

- Accuracy
- Relevance
- Coverage of specialized domains

Example use cases:

- Answering questions from large documents
- Customer support using internal knowledge bases
- Generating reports with current data

**Why not just use an LLM alone?**

- LLMs are trained on fixed datasets. If you ask about a niche domain (like proprietary company manuals or specific coding guidelines), the LLM may not know the answer.
- RAG allows dynamic, up-to-date information retrieval, combining the strengths of search engines and LLMs.


##  **2. Components of a RAG System**

A typical RAG system consists of the following components:

- **Knowledge Base:**
Where your source information is stored. Could be PDFs, text files, or a database.

- **Embeddings Generator:**
Converts documents and queries into numerical vectors that the machine can process.

- **Vector Database (Vector DB):**
Stores embeddings and allows fast similarity searches.

- **Retriever:**
Finds the most relevant documents for a given query by comparing vectors.

- **Large Language Model (LLM):**
Generates the final response using both the retrieved documents and the original query.

- **Indexer / Chunker:**
Prepares the documents for efficient storage and retrieval by splitting them into manageable pieces.


##  **3. Vector Embeddings**

**What are embeddings?**

Embeddings are numerical representations of text in a high-dimensional space.
Similar texts have similar embeddings (close in vector space), while different texts are far apart.

**Why are embeddings used?**

LLMs cannot directly search millions of documents efficiently.
Embeddings let us measure semantic similarity:
“How similar is this document to my query?”
Done via distance metrics like cosine similarity.

<p align="center">
  <img src="images/embeddings.png" width="500">
</p>

In [ ]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
aval_api_key=os.getenv("AVALAI_API_KEY")


In [ ]:
client = OpenAI(    
    api_key=aval_api_key,
    base_url="https://api.avalai.ir/v1"
    )

In [ ]:
res1 = client.embeddings.create(
    input="Python is a programming language.",
    model="text-embedding-3-small"
)

print(len(res1.data[0].embedding))
print(res1.data[0].embedding)

In [ ]:
res2 = client.embeddings.create(
    input="Bananas are yellow fruits.",
    model="text-embedding-3-small"
)

print(len(res2.data[0].embedding))
print(res2.data[0].embedding)

In [ ]:
embeddings = [res1.data[0].embedding, res2.data[0].embedding]
# Convert to numpy arrays for similarity calculations
vecs = [np.array(e) for e in embeddings]

# Example: cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

similarity = cosine_similarity(vecs[0], vecs[1])
print(similarity) 


##  **4. Vector Databases**

**What is a Vector DB?**

- A vector database is a storage system optimized for fast similarity search of embeddings.
- Instead of searching full text, we search vectors to find the most semantically relevant documents.

**Why use them?**

- Efficient for large datasets (millions of documents)
- Fast retrieval with approximate nearest neighbor (ANN) methods
- Simplifies RAG pipelines

**Popular Vector DBs:**

- ChromaDB: Lightweight, Python-friendly, ideal for small/medium projects.
- FAISS (Facebook AI Similarity Search): Optimized for large datasets, extremely fast.
- Pinecone, Weaviate, Milvus: Other options with cloud support.

```
pip install chromadb
```

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.Client()

# Use an embedding function
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=aval_api_key,
    api_base="https://api.avalai.ir/v1",
    model_name="text-embedding-3-small",
    )

collection = client.get_or_create_collection(
    name="my_collection",
    embedding_function=embedding_fn)

# Add documents
collection.add(
    documents=["Python is fun", "Bananas are yellow"],
    metadatas=[{"source": "doc1"}, {"source": "doc2"}],
    ids=["1", "2"],
)


##  **5. Chunking and Indexing**

**Chunking:**

- Large documents are broken into smaller pieces (chunks) so the retriever can handle them efficiently.
- Typical chunk sizes: 200–500 words.
- Helps LLM focus on relevant pieces instead of overwhelming it with a huge document.

**Indexing:**

- Organizes chunks in the vector database.
- Associates each chunk with its embedding and metadata (source, location).
- Makes retrieval fast and precise.

In [ ]:
def chunk_text(text, chunk_size=100):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]


In [ ]:
chunked_data = chunk_text("""A big text:
##  **3. Vector Embeddings**

**What are embeddings?**

Embeddings are numerical representations of text in a high-dimensional space.
Similar texts have similar embeddings (close in vector space), while different texts are far apart.

**Why are embeddings used?**

LLMs cannot directly search millions of documents efficiently.
Embeddings let us measure semantic similarity:
“How similar is this document to my query?”
Done via distance metrics like cosine similarity.

<p align="center">
  <img src="images/embeddings.png" width="500">
</p>
##  **4. Vector Databases**

**What is a Vector DB?**

- A vector database is a storage system optimized for fast similarity search of embeddings.
- Instead of searching full text, we search vectors to find the most semantically relevant documents.

**Why use them?**

- Efficient for large datasets (millions of documents)
- Fast retrieval with approximate nearest neighbor (ANN) methods
- Simplifies RAG pipelines

**Popular Vector DBs:**

- ChromaDB: Lightweight, Python-friendly, ideal for small/medium projects.
- FAISS (Facebook AI Similarity Search): Optimized for large datasets, extremely fast.
- Pinecone, Weaviate, Milvus: Other options with cloud support.

```
pip install chromadb
```
**5. Chunking and Indexing**

**Chunking:**

- Large documents are broken into smaller pieces (chunks) so the retriever can handle them efficiently.
- Typical chunk sizes: 200–500 words.
- Helps LLM focus on relevant pieces instead of overwhelming it with a huge document.

**Indexing:**

- Organizes chunks in the vector database.
- Associates each chunk with its embedding and metadata (source, location).
- Makes retrieval fast and precise.""")

##  **6. Retriever**

**What is a retriever?**
- A retriever is the component that searches the vector DB to find the most relevant chunks for a query.

**Types of retrievers:**
- Dense retrievers: Use embeddings for similarity search (most common in RAG)
- Sparse retrievers: Traditional keyword-based search (e.g., Elasticsearch)

In [ ]:
query = "What is Python?"
results = collection.query(query_texts=[query], n_results=1)
print(results)


### ✅ Key Takeaways:

RAG systems combine retrieval with generation for more accurate, domain-specific responses.

- Core components: embeddings, vector DB, retriever, chunking, LLM.
- Vector databases make large-scale retrieval efficient.
- Chunking and indexing are crucial for performance.
-Building a RAG system involves embedding your documents, storing them in a vector DB, retrieving relevant chunks, and feeding them to an LLM for generation.